In [12]:
 
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [1]:
import zarr
import numpy as np
import networkx as nx

In [8]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]

In [9]:
# annation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017
gt_mask = gt_image >=10000
gt_image= gt_mask * gt_image

In [10]:
np.unique(gt_image)

array([    0, 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016,
       10017], dtype=uint64)

In [6]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)


<Labels layer 'gt_image_2' at 0x7fa048b7bbb0>

In [15]:
from candidate_graph import iterate_tree
import networkx as nx

graph = nx.read_gexf('text.gexf')

def pick_select_edge(graph):
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges

edges, selected_edges = pick_select_edge(graph)

In [18]:
nodes = [nid for nid in graph.nodes()
                if graph.nodes[nid]['t'] == 0]

In [19]:
from linajea_cost_test import get_merge_graph_from_array

t=0
merge_tree = z['Merge_tree/Merge/'+str(t)]

scores  = z['Merge_tree/Scoring/'+str(t)]

merge_tree = get_merge_graph_from_array(merge_tree,scores)
